   # Lab 2

**NETID:SMK220008 | NAME: SRINATH MURALI KRISHNAN**


## Dataset description

The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle. 

The dataset is from the U.S. Small Business Administration (SBA)
The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market 
(SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment.
There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. 
However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.

More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied


In [4]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## The function `describe_more` is data exploration utility

It is up to you if you want to use the function. If you find bugs, please let me know but try to fix them yourself.  
The function most definetely doesn't work with every input

In [6]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:200].to_dict())
                
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

## Load data

We assume data is located in the folder `data` where we run our notebook

In [8]:
data = pd.read_csv('SBA_loans_small.csv')
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,CHGOFF,"$30,000.00","$15,000.00"
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,CHGOFF,"$66,000.00","$33,000.00"
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,CHGOFF,"$50,000.00","$25,000.00"
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,CHGOFF,"$26,000.00","$23,400.00"


In [9]:
data.shape

(448583, 20)

In [10]:
desc_df = describe_more(data)

In [11]:
pd.set_option('display.width', 200)
desc_df.head(n=200)

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,City,24757,object,1,30,"{'LOS ANGELES': 5751, 'HOUSTON': 5102, 'NEW YO..."
1,State,51,object,2,3,"{'CA': 65183, 'TX': 35161, 'NY': 28716, 'FL': ..."
2,Zip,28185,int64,1,5,"{90015: 492, 10001: 455, 93401: 385, 90010: 37..."
3,Bank,5156,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 43347, 'WELLS F..."
4,BankState,54,object,2,3,"{'CA': 58768, 'NC': 39709, 'IL': 32982, 'OH': ..."
5,NAICS,1290,int64,1,6,"{0: 100776, 722110: 13888, 722211: 9713, 81111..."
6,Term,392,int64,1,3,"{84: 114519, 60: 44888, 240: 43020, 120: 38908..."
7,NoEmp,466,int64,1,4,"{1: 76992, 2: 68936, 3: 45269, 4: 36503, 5: 30..."
8,NewExist,3,float64,3,3,"{1.0: 321541, 2.0: 126457, 0.0: 520}"
9,CreateJob,196,int64,1,4,"{0: 313743, 1: 31439, 2: 29216, 3: 14422, 4: 1..."


## Question 1

How many of the each target label (valid values) are there? Target label is `MIS_Status` column. 

In [13]:
desc_df[desc_df['A_Variable']=="MIS_Status"]['Level_Values']

17    {'P I F': 369804, 'CHGOFF': 78779}
Name: Level_Values, dtype: object

## Question 2

Encode target variable `MIS_Status` into `0` and `1`, when value `P I F` is mapped to `0` and `CHGOFF` is mapped to `1`.   

Replace values in the original column

In [15]:
# Assuming your target variable column is named 'target_variable'
# Replace 'value_to_encode_as_1' with the value you want to encode as 1
data['MIS_Status'] = data['MIS_Status'].map({'P I F': 0, 'CHGOFF': 1})


In [16]:
data['MIS_Status'].unique()

array([1, 0])

## Question 3

Think about how to encode `ZIP` column? 

- Should we leave it as is? 
- Should we encode it using one-hot-encoding?
- Should we encode it using target encoder?

#### Question 3a

Copy original Pandas dataframe into new dataframe.  
Encode `ZIP` column using `from sklearn.preprocessing import OneHotEncoder`  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

If you receive memory error, sample the dataset to reduce the number of rows.
Check new dataframe. Is it good idea to use one-hot-encoder on `ZIP` column? 

In [19]:
from sklearn.preprocessing import OneHotEncoder
copy=data
one_hot_encoded = pd.get_dummies(copy['Zip'])
copy = pd.concat([copy, one_hot_encoded], axis=1)


In [20]:
copy.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,0,1,2,3,4,5,6,7,8,9,38,92,98,182,345,417,459,477,501,502,640,711,736,801,820,886,902,985,1001,1002,1004,1005,1006,1007,1008,1009,1010,1012,1013,1014,1017,1020,1022,1024,1025,1026,1027,1028,1029,1030,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1045,1050,1053,1055,1056,1057,1060,1061,1062,1066,1069,1073,1074,1075,1077,1079,1080,1081,1082,1083,1084,1085,1086,1088,1089,1090,1092,1093,1095,1096,1098,1101,1102,1103,1104,1105,1106,1107,1108,1109,1114,1115,1118,1119,1120,1128,1129,1139,1142,1144,1151,1180,1188,1201,1202,1205,1220,1222,1223,1225,1226,1227,1230,1235,1236,1237,1238,1240,1242,1244,1245,1247,1252,1253,1255,1256,1257,1258,1259,1260,1262,1266,1267,1270,1294,1301,1302,1330,1331,1337,1339,1340,1341,1342,1344,1346,1349,1351,1354,1360,1364,1366,1367,1368,1370,1373,1375,1376,1405,1420,1430,1431,1432,1433,1436,1438,1440,1450,1451,1452,1453,1460,1462,1463,1464,1466,1467,1468,1469,1470,1472,1473,1475,1500,1501,1503,1504,1505,1506,1507,1508,1510,1515,1516,1518,1519,1520,1522,1523,1524,1525,1526,1527,1529,1532,1534,1535,1536,1537,1538,1540,1541,1542,1543,1545,1550,1560,1561,1562,1564,1565,1566,1568,1569,1570,1571,1576,1580,1581,1583,1585,1588,1590,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1615,1660,1701,1702,1705,1714,1718,1719,1720,1721,1722,1730,1731,1738,1740,1741,1742,1745,1746,1747,1748,1749,1752,1754,1756,1757,1758,1760,1767,1770,1772,1773,1775,1776,1777,1778,1801,1802,1803,1807,1808,1809,1810,1811,1821,1824,1825,1826,1827,1830,1831,1832,1833,1834,1835,1840,1841,1842,1843,1844,1845,1850,1851,1852,1853,1854,1857,1860,1862,1863,1864,1867,1870,1872,1876,1879,1880,1885,1886,1887,1888,1890,1901,1902,1904,1905,1906,1907,1908,1910,1913,1915,1920,1921,1922,1923,1929,1930,1931,1934,1937,1938,1940,1944,1945,1949,1950,1951,1952,1954,1959,1960,1966,1969,1970,1982,1983,1984,1985,2003,2016,2018,2019,2020,2021,2023,2025,2026,2027,2030,2032,2035,2038,2039,2040,2043,2045,2048,2049,2050,2051,2052,2053,2054,2056,2060,2061,2062,2066,2067,2071,2072,2081,2083,2090,2093,2101,2102,2104,2107,2108,2109,2110,2111,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2148,2149,2150,2151,2152,2154,2155,2156,2158,2159,2160,2161,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2176,2177,2178,2179,2180,2181,2184,2185,2186,2188,2189,2190,2191,2192,2193,2194,2197,2199,2205,2210,2211,2215,2216,2222,2239,2245,2254,2258,2269,2271,2298,2301,2302,2304,2319,2322,2324,2327,2330,2331,2332,2333,2334,2338,2339,2340,2341,2343,2344,2345,2346,2347,2350,2351,2352,2356,2358,2359,2360,2361,2362,2364,2366,2367,2368,2369,2370,2375,2379,2380,2381,2382,2401,2402,2403,2420,2421,2445,2446,2451,2452,2453,2454,2455,2458,2459,2460,2461,2462,2464,2465,2466,2467,2468,2471,2472,2474,2475,2476,2478,2481,2482,2492,2493,2494,2501,2511,2531,2532,2534,2535,2536,2537,2538,2539,2540,2541,2543,2549,2553,2554,2556,2557,2558,2559,2561,2562,2563,2568,2571,2574,2575,2576,2584,2587,2588,2601,2628,2630,2631,2632,2633,2635,2637,2638,2639,2641,2642,2643,2644,2645,2646,2648,2649,2650,2651,2652,2653,2655,2657,2659,2660,2663,2664,2666,2667,2668,2669,2670,2671,2673,2675,2702,2703,2704,2705,2710,2713,2714,2715,2717,2718,2719,2720,2721,2722,2723,2724,2725,2726,2738,2739,2740,2741,2743,2744,2745,2746,2747,2748,2755,2760,2761,2762,2763,2764,2766,2767,2768,2769,2770,2771,2773,2774,2777,2779,2780,2790,2794,2802,2804,2806,2807,2808,2809,2810,2812,2813,2814,2816,2817,2818,2819,2822,2824,2825,2826,2827,2828,2829,2830,2831,2832,2833,2835,2836,2837,2838,2839,2840,2841,2842,2845,...,97526,97527,97530,97532,97535,97536,97537,97539,97540,97541,97544,97587,97601,97603,97607,97620,97622,97624,97625,97626,97630,97632,97640,97643,97660,97684,97701,97702,97707,97708,97709,97710,97711,97720,97730,97731,97

Zipcode has a large number of unique values which are non ideal for one hot encoding as it leads to a high number id columns. This can significantly increase the dimensionality of the dataset, which may lead to the curse of dimensionality and make the model more complex as observed in this case.


### Question 3b

Encode ZIP column using target **(mean)** encoding technique.
At this point `MIS_Status` column contains only 0 and 1.
Replace `ZIP` values with the mean of the response column (`MIS_Status`) for each value in a `ZIP` column. 

For example, if for ZIP=11222 there are total of 150 records. Out of which 80 records have label `0` and 70 records have label `1`.   

Populate new column value `ZIP_TE` for the records with `ZIP=11222` with value:  
(0x80 + 1x70)/(80+70) = 0.467


Since you're using the target variable to create new features, there's a risk of over-fitting, especially if categories have a very low number of observations. One way to mitigate this is to use a form of regularization, like adding a smoothing parameter. The library `category_encoders` has a `TargetEncoder` class that implements this functionality.

In [23]:
import category_encoders as ce

# Assuming your dataframe is named 'df' and the response column is named 'MIS_Status'
# Assuming the ZIP column is named 'ZIP'

# Create a TargetEncoder instance
encoder = ce.TargetEncoder(cols='Zip', smoothing=10)  # You can adjust the smoothing parameter as needed

# Fit and transform the data
data['ZIP_TE'] = encoder.fit_transform(data['Zip'], data['MIS_Status'])

# Now df['ZIP_TE'] contains the target encoded values for the ZIP column

In [24]:
data.head()

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,ZIP_TE
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,N,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113705
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,N,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080883
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,N,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.154699
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,N,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247119
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,N,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049646


## Question 4 - replace the column with categorical

Encode `LowDoc` column using `from sklearn import preprocessing.LabelEncoder`. Replace original column. 

In [26]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()
data['LowDoc'] = label_encoder.fit_transform(data['LowDoc'])

## Question 5

Fill all columns with missing values with `Unknown` or equivalent value.

```
# filling with Unknown class
df_clean = df.fillna("Unknown")
df_clean
```

In [28]:
for col in data.columns:
    if data[col].isna().any() == True:
        print(col)

City
State
Bank
BankState
NewExist
RevLineCr


In [29]:
data_clean = data.fillna("Unknown")
data_clean

,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv,ZIP_TE
0,BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1.0,0,8,1,1,Y,4,"$60,639.00",$0.00,1,"$30,000.00","$15,000.00",0.113705
1,SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1.0,0,2,1,1,0,4,"$66,000.00",$0.00,1,"$66,000.00","$33,000.00",0.080883
2,PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1.0,0,0,1,1,N,4,"$50,000.00",$0.00,1,"$50,000.00","$25,000.00",0.154699
3,HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1.0,1,3,0,1,Y,4,"$168,853.00",$0.00,1,"$50,000.00","$25,000.00",0.247119
4,LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2.0,0,0,0,0,N,4,"$26,000.00",$0.00,1,"$26,000.00","$23,400.00",0.049646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448578,HUNTINGTON BEACH,CA,92649,WELLS FARGO BANK NATL ASSOC,SD,0,300,10,1.0,0,0,1,0,N,4,"$492,000.00",$0.00,0,"$492,000.00","$344,400.00",0.125001
448579,DOVER,NH,3820,CITIZENS BANK NATL ASSOC,RI,812112,36,1,2.0,0,1,0,1,N,4,"$10,000.00",$0.00,0,"$10,000.00","$5,000.00",0.081967
448580,LUBBOCK,TX,79423,PLAINSCAPITAL BANK,TX,611691,64,1,2.0,1,1,52007,1,N,4,"$68,000.00",$0.00,0,"$68,000.00","$57,800.00",0.080843
448581,SULLIVAN,IL,61951,FIRST MID-ILLINOIS BK & TRUST,IL,0,60,2,1.0,0,0,1,0,N,7,"$23,200.00",$0.00,0,"$23,200.00","$18,560.00",0.159508


## Question 6

Load same `SBA_loans_small.zip` into H2O-3 dataframe.  
Describe imported dataframe.

In [31]:
import h2o
#start h2o cluster with limited memory and cpu
h2o.init(max_mem_size = "2G", nthreads = 2)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 days 23 hours 57 mins
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_srinathmurali_793rhf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.648 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [32]:
df_h = h2o.import_file(
  path="SBA_loans_small.csv",
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


## Question 7

Use appropriate  method to fill missing values for the following columns:
- City
- State
- Bank
- BankState
- NewExist
- RevLineCr
- LowDoc

Use `fillna` method to fill missing values for numerical variables.

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-munging/fillnas.html

In [34]:
'''All Dataframe columns '''
df_h.columns

['City',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'DisbursementGross',
 'BalanceGross',
 'MIS_Status',
 'GrAppv',
 'SBA_Appv']

In [35]:
df_h

City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",0,CHGOFF,"$30,000.00","$15,000.00"
SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1,0,2,1,1,,N,"$66,000.00",0,CHGOFF,"$66,000.00","$33,000.00"
PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",0,CHGOFF,"$50,000.00","$25,000.00"
HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1,1,3,0,1,Y,N,"$168,853.00",0,CHGOFF,"$50,000.00","$25,000.00"
LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",0,CHGOFF,"$26,000.00","$23,400.00"
CANTON,OH,44708,APPLE CREEK BK. CO,OH,722211,32,2,2,6,2,0,1,N,N,"$75,000.00",0,CHGOFF,"$75,000.00","$67,500.00"
Idaho Falls,ID,83404,"READYCAP LENDING, LLC",CA,811118,100,25,2,0,25,0,1,N,N,"$603,000.00",0,CHGOFF,"$623,000.00","$467,250.00"
TAYLOR,MI,48180,CITIZENS BANK NATL ASSOC,RI,811191,54,3,1,0,3,0,1,Y,N,"$300,000.00",0,CHGOFF,"$100,000.00","$50,000.00"
MCALLEN,TX,78502,INTER NATIONAL BANK,TX,0,79,6,1,0,0,0,0,N,N,"$400,000.00",0,CHGOFF,"$400,000.00","$320,000.00"
PHOENIX,AZ,85238,COMMUNITY BANK OF NEVADA,NV,722211,4,7,1,0,7,80025,1,Y,N,"$77,820.00",0,CHGOFF,"$50,000.00","$25,000.00"


In [36]:
categorical_columns = ['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc']  # Add other categorical columns as needed
df_h['City'] = df_h['City'].isna().ifelse("missing", df_h['City'])
df_h['State'] = df_h['State'].isna().ifelse("missing", df_h['State'])
df_h['Bank'] = df_h['Bank'].isna().ifelse("missing", df_h['Bank'])
df_h['BankState'] = df_h['BankState'].isna().ifelse("missing", df_h['BankState'])
df_h['RevLineCr'] = df_h['RevLineCr'].isna().ifelse("missing", df_h['RevLineCr'])
df_h['LowDoc'] = df_h['LowDoc'].isna().ifelse("missing", df_h['LowDoc'])

In [37]:
numeric_columns = ['NewExist']
df_h['NewExist']=df_h['NewExist'].fillna(method="forward",axis=0,maxlen=1)

In [38]:
df_h['NewExist'].isna().any()

False

In [39]:
df_h

City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,MIS_Status,GrAppv,SBA_Appv
BOULDER,CO,80302,WELLS FARGO BANK NATL ASSOC,SD,327122,39,8,1,0,8,1,1,Y,N,"$60,639.00",0,CHGOFF,"$30,000.00","$15,000.00"
SEATTLE,WA,98101,HANMI BANK,CA,722211,15,2,1,0,2,1,1,missing,N,"$66,000.00",0,CHGOFF,"$66,000.00","$33,000.00"
PHOENIX,AZ,85023,CAPITAL ONE NATL ASSOC,VA,323119,62,15,1,0,0,1,1,N,N,"$50,000.00",0,CHGOFF,"$50,000.00","$25,000.00"
HOUSTON,TX,77074,JPMORGAN CHASE BANK NATL ASSOC,IL,524210,66,3,1,1,3,0,1,Y,N,"$168,853.00",0,CHGOFF,"$50,000.00","$25,000.00"
LAS CRUCES,NM,88001,BANK OF AMERICA NATL ASSOC,NM,0,60,1,2,0,0,0,0,N,N,"$26,000.00",0,CHGOFF,"$26,000.00","$23,400.00"
CANTON,OH,44708,APPLE CREEK BK. CO,OH,722211,32,2,2,6,2,0,1,N,N,"$75,000.00",0,CHGOFF,"$75,000.00","$67,500.00"
Idaho Falls,ID,83404,"READYCAP LENDING, LLC",CA,811118,100,25,2,0,25,0,1,N,N,"$603,000.00",0,CHGOFF,"$623,000.00","$467,250.00"
TAYLOR,MI,48180,CITIZENS BANK NATL ASSOC,RI,811191,54,3,1,0,3,0,1,Y,N,"$300,000.00",0,CHGOFF,"$100,000.00","$50,000.00"
MCALLEN,TX,78502,INTER NATIONAL BANK,TX,0,79,6,1,0,0,0,0,N,N,"$400,000.00",0,CHGOFF,"$400,000.00","$320,000.00"
PHOENIX,AZ,85238,COMMUNITY BANK OF NEVADA,NV,722211,4,7,1,0,7,80025,1,Y,N,"$77,820.00",0,CHGOFF,"$50,000.00","$25,000.00"


## Question 8

Perform target encoding on the following columns:
- ZIP
- City
- State

https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/target-encoding.html


In [41]:
from h2o.estimators import H2OTargetEncoderEstimator

In [42]:
df_h['MIS_Status'] = df_h['MIS_Status'].asfactor()
response='MIS_Status'

# Split the dataset into train and test
train, test = df_h.split_frame(ratios = [.8], seed = 1234)

# Choose which columns to encode
encoded_columns = ["Zip", "City", "State"]

# For k_fold strategy we need to provide fold column
fold_column = "kfold_column"
train[fold_column] = train.kfold_column(n_folds=5, seed=1234)

# Train a TE model
filled_te = H2OTargetEncoderEstimator(fold_column=fold_column,
                                       data_leakage_handling="k_fold",
                                       blending=True,
                                       inflection_point=3,
                                       smoothing=10,
                                       noise=0.15,     # In general, the less data you have the more regularization you need
                                       seed=1234)

filled_te.train(x=encoded_columns,
                 y=response,
                 training_frame=train)

# New target encoded train and test sets
train_te = filled_te.transform(frame=train, as_training=True)
test_te = filled_te.transform(frame=test, noise=0)

targetencoder Model Build progress: |████████████████████████████████████████████| (done) 100%
